In [15]:
using JuMP
import LinearAlgebra
import SCS
import Random

In [16]:
N, d = 2, 2

(2, 2)

In [17]:
Random.seed!(7);
q = rand(N)
sumq = sum(q)
q = q / sumq

2-element Vector{Float64}:
 0.25316816205570525
 0.7468318379442948

In [18]:
function random_state(d)
    x = randn(ComplexF64, (d, d))
    y = x * x'
    return LinearAlgebra.Hermitian(y / LinearAlgebra.tr(y))
end

ρ = [random_state(d) for i in 1:N]

2-element Vector{LinearAlgebra.Hermitian{ComplexF64, Matrix{ComplexF64}}}:
 [0.855925063606513 + 0.0im 0.267624920054898 - 0.21908924768105717im; 0.267624920054898 + 0.21908924768105717im 0.1440749363934869 + 0.0im]
 [0.2165426668737068 + 0.0im 0.14818042420981853 + 0.265768601481561im; 0.14818042420981853 - 0.265768601481561im 0.7834573331262932 + 0.0im]

In [19]:
model = Model(SCS.Optimizer)
set_silent(model)

In [20]:
E = [@variable(model, [1:d, 1:d] in HermitianPSDCone()) for i in 1:N]

2-element Vector{LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[4] im; _[2] - _[4] im _[3]]
 [_[5] _[6] + _[8] im; _[6] - _[8] im _[7]]

In [21]:
E0 = @variable(model, [1:d, 1:d] in HermitianPSDCone())

2×2 LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}:
 _[9]              _[10] + _[12] im
 _[10] - _[12] im  _[11]

In [22]:
@constraint(model, (sum(E) + E0) == LinearAlgebra.I)

[_[1] + _[5] + _[9] - 1                              …  _[2] + _[6] + _[10] + _[4] im + _[8] im + _[12] im
 _[2] + _[6] + _[10] - _[4] im - _[8] im - _[12] im     _[3] + _[7] + _[11] - 1] ∈ Zeros()

In [23]:
for i in 1:N
    @constraint(model, (q[i] * real(LinearAlgebra.tr(ρ[i] * E[i]))) == 0)
end

In [24]:
@objective(
    model,
    Max,
    sum(q[i] * real(LinearAlgebra.tr(ρ[i] * E[i])) for i in 1:N) / N,
)

0.10834648760533676 _[1] + 0.06775410913060358 _[2] - 0.055466422161580424 _[4] + 0.01823759342251586 _[3] + 0.0808604789473248 _[5] + 0.11066585855998404 _[6] + 0.19848445311235902 _[8] + 0.2925554400248226 _[7]

In [25]:
optimize!(model)
assert_is_solved_and_feasible(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 1.05903e-07
  Dual objective value : -9.78062e-06

* Work counters
  Solve time (sec)   : 2.48401e-02


In [26]:
objective_value(model)

1.0590291541516092e-7

In [27]:
solution = [value.(e) for e in E]

2-element Vector{Matrix{ComplexF64}}:
 [2.6488240709311398e-5 + 0.0im -5.30507296105371e-5 + 3.280598885917063e-5im; -5.30507296105371e-5 - 3.280598885917063e-5im 0.00014115759539897076 + 0.0im]
 [2.675854534698188e-5 + 0.0im 3.1382937663447855e-5 - 8.296559222208811e-6im; 3.1382937663447855e-5 + 8.296559222208811e-6im -1.337980426469785e-5 + 0.0im]

In [28]:
value.(E0)

2×2 Matrix{ComplexF64}:
   0.999947+0.0im         2.14201e-5-2.46498e-5im
 2.14201e-5+2.46498e-5im    0.999872+0.0im